# Introduction to Convolutional Neural Networks

This tutorial addresses the basic concepts regarding Convolutional Neural Networks and their implementation using the Keras framework.
Convolutional Neural Networks (CNNs) are a class of feed-forward artificial neural architecture. They are applied to analyse visual 2D imagery, meaning that we can feed images directly into a CNN without the need to flatten them into a 1D vector as done in the previous tutorial.
CNNs have revolutionised the field of computer vision in the last decade. In 2012 Alex Krizhevsky introduced the AlexNet architecture to win the ImageNet Challenge (one of the most important competitions on image classification within the Computer Vision community), by reducing the top-5 error more than 10 percentage points, which was an incredible improvement at that time. As of now, CNNs are used not only on image classification but in many other computer vision tasks.

![](https://cdn-5f733ed3c1ac190fbc56ef88.closte.com/wp-content/uploads/2017/03/alexnet_small-1.png)

The image above is from [cv-tricks' blog](https://cv-tricks.com/cnn/understand-resnet-alexnet-vgg-inception/) and shows the proposed AlexNet architecture. It is composed of 5 convolutional layers followed by 3 fully connected layers. Nowadays, we can find much deeper and more complex architectures, which outperform AlexNet on the ImageNet Challenge.







## CNN Structure

The basic pipeline of common CNNs consists of an image as input and a stack of convolutional layers that extract a feature representation from the input image. The final shape of the image representation is conditioned on the type of problem/task that the architecture is facing. For instance, the output of the last layer in a classification problem is a probability vector. Each dimension of the probability vector represents how likely is that the input image belongs to a specific class. However, the architecture design is up to us, and therefore, we could code a network that outputs a single value for regression problems, or that generates a new image map for semantic segmentation. Now, let's dig in a bit into CNN and introduce some layers that are widely used.

### 2D Convolutional Layer

The most common layer in any CNN architecture is the 2D convolutional layer. Convolutional layers are specifically designed to extract features from images or even extract features from previously extracted features. As shown in the following illustration, 2D convolutions apply the same filter to the full image. And therefore, due to its nature, 2D convolutional filters exploit the local information presented in images, making them a powerful tool for image analysis.

![](https://cdn-images-1.medium.com/max/800/1*Fw-ehcNBR9byHtho-Rxbtw.gif)

Image [source](https://towardsdatascience.com/intuitively-understanding-convolutions-for-deep-learning-1f6f42faee1).

The latest deep learning frameworks have made possible the integration of convolutional layers easily on our architectures with only a single line of code. We will address here how 2D convolutions work since full understanding is needed to comprehend how any CNN operates. The following images and some explanations can be further explored on the original [Irhum Shafkat's blog](https://towardsdatascience.com/intuitively-understanding-convolutions-for-deep-learning-1f6f42faee1) or in the [Neural Networks and Deep Learning](http://neuralnetworksanddeeplearning.com/) book. Both are strongly recommended.

To understand 2D convolutions, we need to define first what a kernel is. Kernels are simply matrices of numbers. The numbers on the kernels are the so-called weights, and the weights on the kernels change as we train the network. Hence, network training aims to optimise the network's parameters (update the kernel weights) such that the cost function is minimised.

The 2D convolution operation takes the network's kernels and "slides" them over the input image (alike to a sliding window) as in the following image from [PyImageSearch](https://www.pyimagesearch.com/2015/03/23/sliding-windows-for-object-detection-with-python-and-opencv/) blog:

![](https://pyimagesearch.com/wp-content/uploads/2014/10/sliding_window_example.gif)

In each step, the network performs an element-wise multiplication with the elements that are currently on. The results of this elementwise multiplication are added to obtain the output value of the operation. CNNs repeat previous step for all the positions of the sliding window, composing at the end the feature map. This generated feature map can go through another 2D convolutional layer and create more powerful features.

>

![](https://cdn-images-1.medium.com/max/800/1*Zx-ZMLKab7VOCQTxdZ1OAw.gif)

>
The previous image shows the 2D convolution operation. Thus, the new feature values are the weighted sum of all the elements in the sliding window after the elementwise multiplication between input and kernel.  The bigger the size of the kernel is, the more feature elements contribute to the final output value. In contrast to fully connected layers, where a new feature value is a weighted sum over **all** input values, as mentioned, 2D convolutions compute features based on local areas. In other words, instead of looking at every input component, they consider only features coming from close locations.

In the above example, the input image on the left has a size of 5x5 and the dimension of the resulting feature map is 3x3, showing that the size of the output maps is not always equal to the input. Hence, the output size can be computed by doing:

$O = W - K + 1$,

where $O$ is the output height/length, $W$ is the input height/length and $K$ is the kernel size. The output size is not only conditioned on the input size but also on the kernel size. Check in the following code cell how the output feature map shape changes as you increase the kernel size. In Pytorch, we define the layer by using `Conv2D` from `torch.nn` (documentation [here](https://docs.pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)).









In [ ]:
import torch
import torch.nn as nn
import numpy as np

# Generate dummy data (same as Keras input shape: (1, 100, 100, 1))
input_feature = np.random.random((1, 1, 100, 100)).astype(np.float32)  # PyTorch expects (N, C, H, W)
input_tensor = torch.from_numpy(input_feature)

# Define a simple model with a single 2D convolutional layer
class SimpleConvNet(nn.Module):
    def __init__(self):
        super(SimpleConvNet, self).__init__()
        self.conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(3, 3))

    def forward(self, x):
        return self.conv(x)

# Instantiate and apply the model
model = SimpleConvNet()
output_tensor = model(input_tensor)

# Convert output to numpy for shape comparison
output_feature = output_tensor.detach().numpy()

print('Input size: ({:}, {:})'.format(input_feature.shape[2], input_feature.shape[3]))
print('Output size: ({:}, {:})'.format(output_feature.shape[2], output_feature.shape[3]))


Moreover, the kernel size and the input size are not the only parameters affecting the output size. We are going to introduce two extra elements that change the size of the output map: the padding and the stride.

### Adding Padding to Input Features

In some tasks, such as [image translation](https://arxiv.org/pdf/1611.07004.pdf), we need the output size to be equal to the input size. The solution to that is using padding, where extra edges are added to the input features so that the dimension is not reduced after the convolutional layer. Normally those pixels have $0$ value (termed zero-padding), but depending on the application other methods could be used, e.g., reflection or symmetric padding.

![](https://cdn-images-1.medium.com/max/800/1*1okwhewf5KCtIPaFib4XaA.gif)

If padding is used, the new output size can be computed by doing:

$O = W - K + 2P+ 1$,

where $P$ is the padding value. $P$ must be set in concordance with the kernel size if dimensionality wants to be preserved. Padding can be added to the `Conv2d` layer in Keras by using the padding argument and selecting one of the two settings: `valid`, which is the default value and means no padding; and `same`, which adapts the padding value to have the same output size as input size.

In [ ]:
import torch
import torch.nn as nn
import numpy as np

# Generate dummy data
input_feature = np.random.random((1, 1, 100, 100)).astype(np.float32)  # (N, C, H, W)
input_tensor = torch.from_numpy(input_feature)

# Define model with 'same' padding
class SamePadConvNet(nn.Module):
    def __init__(self):
        super(SamePadConvNet, self).__init__()
        # For kernel_size=3 and stride=1, padding=1 achieves 'same' output size
        self.conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, padding=1)

    def forward(self, x):
        return self.conv(x)

# Instantiate and apply the model
model = SamePadConvNet()
output_tensor = model(input_tensor)

# Convert output to numpy
output_feature = output_tensor.detach().numpy()

print('Input size: ({:}, {:})'.format(input_feature.shape[2], input_feature.shape[3]))
print('Output size: ({:}, {:})'.format(output_feature.shape[2], output_feature.shape[3]))


### Using Stride in Convolutional Layers

The stride operation allows the convolutional layers to skip some of the sliding windows explained above. Hence, instead of jumping one pixel apart, we can define the number of skipped elements before computing the weighting sum between the kernel's weights and input features. A stride of 1 means that features will be extracted from all windows a pixel apart, so basically, every single window is computed. A stride of 2 means that we are selecting windows 2 pixels apart, skipping every other window in the process. Strides reduce the number of computations and consequently the size of the output map. In practice, as we go deeper into the CNN, the spatial size of the feature map gets smaller while the number of channels increases. Moreover, we can further reduce the size of the feature map using pooling operations, which we introduce later in this tutorial.

![](https://cdn-images-1.medium.com/max/800/1*BMngs93_rm2_BpJFH2mS0Q.gif)

If strides are used, the new output size can be computed as:

$O = \dfrac{W - K + 2P}{S}+ 1$,

where $S$ is the stride value. The stride is set in the layer by using the `strides` argument.

In [ ]:
import torch
import torch.nn as nn
import numpy as np

# Generate dummy data
input_feature = np.random.random((1, 1, 100, 100)).astype(np.float32)  # Shape: (N, C, H, W)
input_tensor = torch.from_numpy(input_feature)

# Define model with 'same' padding and stride=2
class StridedSamePadConvNet(nn.Module):
    def __init__(self):
        super(StridedSamePadConvNet, self).__init__()
        # With kernel_size=3 and stride=2, padding=1 keeps output size as ceil(input/stride)
        self.conv = nn.Conv2d(
            in_channels=1,
            out_channels=1,
            kernel_size=3,
            stride=2,
            padding=1  # This simulates Keras's 'same' padding with stride=2
        )

    def forward(self, x):
        return self.conv(x)

# Instantiate and apply the model
model = StridedSamePadConvNet()
output_tensor = model(input_tensor)

# Convert output to numpy
output_feature = output_tensor.detach().numpy()

print('Input size: ({:}, {:})'.format(input_feature.shape[2], input_feature.shape[3]))
print('Output size: ({:}, {:})'.format(output_feature.shape[2], output_feature.shape[3]))


## Differences Between Kernel and Filters

The examples above take as input a single-channel image and compute a feature map with also one channel. However, when dealing with RGB images or feature maps, the input is no longer a single-channel map but, instead, they can have multiple channels. In the case of an RGB image, for each 2D convolution, we will need to define 3 kernels to interact with each of the image's channel colours. This group of kernels is called a filter. Thus, a filter is a collection of kernels that produces a single output.

As a regular practice when defining Deep Learning models, we increase the number of filters in each convolutional layer as we go deeper into the model. Due to the element-wise multiplication, the number of kernels on each filter must be the same that the number of channels in the input feature map. Keras already deals with the number of kernels inside each filter. Keras keeps track of the input size in each convolutional layer unlike other frameworks (Pytorch or TensorFlow). Hence, in Keras, we must only decide the number of filters (output channels) in each layer.

The next figure shows how the convolution is performed when having three input channels. First, one filter uses its three independent kernels to convolve with the RGB channels of the input image:

![](https://cdn-images-1.medium.com/max/1000/1*8dx6nxpUh2JqvYWPadTwMQ.gif)

Next, each of the processed feature maps is added together to obtain a single channel:

![](https://cdn-images-1.medium.com/max/1000/1*CYB2dyR3EhFs1xNLK8ewiA.gif)

Finally, we add the bias term to obtain the feature map. There is a single bias for the full output channel map. This operation is repeated for all the filters inside the convolutional layer.

Now, we show how to use a `Conv2D` layer that takes an input image with 3 channels and generates an output map with 32 channels.


In [ ]:
import torch
import torch.nn as nn
import numpy as np

# Generate dummy data: Keras shape (1, 100, 100, 3) → PyTorch shape (1, 3, 100, 100)
input_feature = np.random.random((1, 3, 100, 100)).astype(np.float32)
input_tensor = torch.from_numpy(input_feature)

# Define the PyTorch model
class StridedConvNet(nn.Module):
    def __init__(self):
        super(StridedConvNet, self).__init__()
        self.conv = nn.Conv2d(
            in_channels=3,        # 3 input channels (RGB)
            out_channels=32,      # 32 filters
            kernel_size=3,
            stride=2,
            padding=1             # 'same' padding for 3x3 kernel with stride 2
        )

    def forward(self, x):
        return self.conv(x)

# Instantiate and run the model
model = StridedConvNet()
output_tensor = model(input_tensor)

# Convert to numpy for shape inspection
output_feature = output_tensor.detach().numpy()

# Print input and output sizes
print('Input size: ({:}, {:}, {:})'.format(input_feature.shape[2], input_feature.shape[3], input_feature.shape[1]))
print('Output size: ({:}, {:}, {:})'.format(output_feature.shape[2], output_feature.shape[3], output_feature.shape[1]))


## Activation Functions

As seen in previous tutorials, after a `Linear` layer we usually can find an activation function. We introduce here how to use them after `Conv2D` layers. Those activation functions are a set of operators that maps the feature values to a new set of values, the mapping depends on the function at hand. The main reason for using activation functions is that they add non-linearities to the network, giving more expressive power to the network, which will be able to reproduce more complex functions.


The complete list of activation functions that Keras offers can be found [here](https://docs.pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity). We introduce here some of them:

*  **Sigmoid Function** sets the output in the range (0, 1). The sigmoid function is widely used in binary classification problems since its output can be taken as a probability value. `nn.Sigmoid(x)`:

>![](https://i.ibb.co/Ph8dsTv/sigmoid.png)

*  **Tanh Function** is a logistic function as sigmoid, but the range of the tanh function is (-1, 1). Contrary to sigmoid function, where the values close to 0 are set around 0.5, in the tanh function they will be still mapped around the 0 value. `nn.Tanh(x)`:

>![](https://i.ibb.co/68g7LpL/tanh.png)

*  **ReLU Function** is the most common activation function you can find in any current CNN as in general works better than the rest. The range of this function is in \[0, inf). It sets all negative values to 0 and hence is computationally easy to implement. As a drawback, during training some neurons *die*, meaning that the output is 0 for all available data points and no gradient is propagated there. `nn.ReLU(x)`:

>![](https://i.ibb.co/Zd9H8Z4/relu.png)

*  **LeakyReLU Function** is a modified version of the ReLU activation above, which attempts to solve the problem of dying neurons that ReLU has. While ReLU does not backpropagate negative values, Leaky ReLU smooths those values without setting them to 0. That allows the gradients to backpropagate through the network even for negative values. `nn.LeakyReLU(negative_slope=0.3)`:

>![](https://i.ibb.co/dmnJ6h1/leakyrelu.png)

*  **Softmax Function** is another widely activation function for multi-class classification problems and usually is employed as the last activation function in the classification model. This function sets all of the output elements to the range (0, 1). However, the softmax function does not take independently the input values to map it into its probability value. Softmax Function takes an un-normalized vector, $s$, and normalizes it into a probability distribution, $p$, following the softmax expression. As the output is a probability, the output elements add up to 1. `nn.Softmax(x, axis=-1)`. Thus, the output value $p_i$ is computed as:

> $p_{i} = \dfrac{e^{s_i}}{\sum_{\substack{j}}^{N} e^{s_j}}$

The following example shows the feature maps before and after of the ReLU activation function. All values that are negative are set to 0 after the activation function.




In [ ]:
import torch
import torch.nn as nn
import numpy as np

# Generate dummy data: (batch, channels, height, width)
input_feature = (np.random.random((1, 1, 3, 3)) - 0.5).astype(np.float32)
input_tensor = torch.from_numpy(input_feature)

# Define model
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv = nn.Conv2d(
            in_channels=1,
            out_channels=1,
            kernel_size=3,
            stride=1,
            padding=1,  # 'same' padding for 3x3 kernel
        )
        self.relu = nn.ReLU()

    def forward(self, x):
        conv_out = self.conv(x)
        relu_out = self.relu(conv_out)
        return conv_out, relu_out  # returning both

# Instantiate model
model = ConvNet()

# Run model
with torch.no_grad():
    output_feature, output_relu_feature = model(input_tensor)

# Convert to numpy for printing
output_feature = output_feature.numpy()
output_relu_feature = output_relu_feature.numpy()

# Print results
print('Output Network without activation function')
print(output_feature[0, 0])  # print only the single channel

print('\nOutput Network after ReLU activation function')
print(output_relu_feature[0, 0])

## Pooling Layer

It is a common practice to insert a pooling layer between convolutional layers in CNNs. In a standard CNN architecture, we set the feature sizes to become smaller progressively to reduce the computation in the networks, and to merge the information from different spatial locations. To reduce the feature map sizes, we can either use bigger stride size in the convolutional layers or we can use pooling layers. Pooling layers perform a spatial sliding window and apply an operation to reduce the spatial size. Those operations vary depending on the architecture, being the max, mean and min pooling the most typical ones. Here, we will explain the max pooling, although all the others work similarly. Max pooling keeps only the max value in a neighbourhood, where the neighbourhood is defined by the size of the kernel. Let's visualise it, the next example shows the result of a Max Pooling layer with a 2x2 kernel and a stride of 2.

![](https://i.ibb.co/Xp454S4/MaxPool.png)

As in convolutional layers, the final size is conditioned to the stride size of the pooling layer. However, contrary to convolutional layers, pooling layers operate independently on each of the input channels, without modifying the depth of the feature maps. To use max pooling in our model we use, `nn.MaxPooling2d` and define the stride and pooling size.

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

# Generate dummy data
input_feature = np.random.random((1, 3, 100, 100)).astype(np.float32)
input_tensor = torch.from_numpy(input_feature)

# Define the model
# input: 100x100 image with 3 channels -> (100, 100, 3) tensor.
# this applies 32 convolution filters of size 3x3 each.
# attribute padding='same' applies zero-padding to the input feature map
# attribute strides=1 applies applies stride of 1
class ConvMaxPoolNet(nn.Module):
    def __init__(self):
        super(ConvMaxPoolNet, self).__init__()
        self.conv = nn.Conv2d(
            in_channels=3,
            out_channels=32,
            kernel_size=3,
            stride=1,
            padding=1  # 'same' padding
        )
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(
            kernel_size=2,
            stride=2,
            padding=1  # emulate 'same' padding
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        x = self.pool(x)
        return x

# Instantiate and run the model
model = ConvMaxPoolNet()
with torch.no_grad():
    output_tensor = model(input_tensor)

# Convert output to numpy
output_feature = output_tensor.numpy()

# Print input and output sizes
print('Input size: ({:}, {:}, {:})'.format(input_feature.shape[2], input_feature.shape[3], input_feature.shape[1]))
print('Output size: ({:}, {:}, {:})'.format(output_feature.shape[2], output_feature.shape[3], output_feature.shape[1]))


# Example: Classification on MNIST

As opposed to the *Introduction to Keras* tutorial, in this section, we show how to perform image classification when the input data is a 2D image instead of a flat 1D vector.

As discussed above, Convolutional Neural Networks aim to extract and exploit the local relationships on 2D maps, hence, CNNs are much more convenient for images than Multi-layer Perceptron models.

First of all, we load the MNIST dataset from Keras' framework. The definition of the data is almost identical than in the Keras tutorial, although this time we are not reshaping the input images into a 1D vector.

In [ ]:
import numpy as np
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, random_split, TensorDataset

# the data, shuffled and split between train and test sets
# Here we are using the official test set as our validation set, in further
# tutorials, test and validation splits will be explained properly.
# Hence, even though the variables are `x_test` and `y_test`, they represent our validation set

transform = transforms.Compose([
    transforms.ToTensor(),  # converts HxWxC in [0,255] to CxHxW in [0.0,1.0]
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
val_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Convert datasets to numpy arrays
x_train = train_dataset.data.numpy().astype('float32')
x_test = val_dataset.data.numpy().astype('float32')
y_train = train_dataset.targets.numpy()
y_test = val_dataset.targets.numpy()
shape = x_train.shape

# Normalize and reshape the input images
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

x_train /= 255
x_test /= 255

y_train_class = np.eye(10)[y_train]  # one-hot encoding
y_test_class = np.eye(10)[y_test]    # one-hot encoding

# Convert data to PyTorch tensors
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).permute(0, 3, 1, 2)  # (N, C, H, W)
y_train_tensor = torch.tensor(y_train_class, dtype=torch.float32)

x_test_tensor = torch.tensor(x_test, dtype=torch.float32).permute(0, 3, 1, 2)
y_test_tensor = torch.tensor(y_test_class, dtype=torch.float32)

# Create dataloaders
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


print()
print('Image shape: {0}'.format(x_train.shape[1:]))
print('Total number of training samples: {0}'.format(x_train.shape[0]))
print('Total number of validation samples: {0}'.format(x_test.shape[0]))

As illustrated in the cell, this time the input image before the model is 28x28x1.

Now we can define a model composed of convolutional layers, activation functions, and maxpool operators:

In [ ]:
import torch
import torch.nn as nn

class CNNModel(nn.Module):
    def __init__(self, input_channels=1):
        super(CNNModel, self).__init__()
        self.model = nn.Sequential(
            # input shape: (batch_size, input_channels, H, W)
            nn.Conv2d(in_channels=input_channels, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),

            nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

    def forward(self, x):
        return self.model(x)

As explained in the previous tutorial, in a classification problem, the output of the model is a probability vector. Each dimension of the vector indicates how likely is that the input image belongs to a specific class.

Up to now, the resulting feature map of the model is a map with the shape *Batch x Weight' x Height' x Channel*, and it needs to be mapped into a vector with shape *Batch x Num Classes*. A common technique to process this mapping is to add a Flatten layer that will reshape the feature map to *Batch x (Weight' * Height' * Channel)*. Following the Flatten layer, we add a dense layer which maps this new feature map to the desired output size.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class ClassificationHead(nn.Module):
    def __init__(self, input_features):
        super(ClassificationHead, self).__init__()
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(input_features, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc(x)
        x = F.softmax(x, dim=1)  # Apply softmax across classes
        return x


In [ ]:
model = nn.Sequential(
    CNNModel(input_channels=1),
    ClassificationHead(input_features=16 * 7 * 7)
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

Let's visualize the output shape in each layer.

In [ ]:
from torchsummary import summary
import torch

summary(model, input_size=(1, 28, 28))

Finally, we can train our CNN and check its performance on MNIST.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=1e-4)

# Training loop
for epoch in range(10):
    model.train()
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        targets_class = torch.argmax(targets, dim=1)  # convert one-hot to class index

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets_class)
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
correct = 0
total = 0
loss_total = 0

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        targets_class = torch.argmax(targets, dim=1)

        outputs = model(inputs)
        loss = criterion(outputs, targets_class)
        loss_total += loss.item() * inputs.size(0)

        preds = torch.argmax(outputs, dim=1)
        correct += (preds == targets_class).sum().item()
        total += inputs.size(0)

avg_loss = loss_total / total
accuracy = correct / total

print('Validation loss:', avg_loss)
print('Validation accuracy:', accuracy)

# Coursework


## Task 1: Classification

At this point, we know what is a CNN, how they work, and the components needed to design them. In this first task, we want you to create a CNN that is able to outperform the Multi-layer Perceptron model from Tutorial 1. For the first part of the coursework, we train on CIFAR10, a  classical dataset for image classification. Note that in these tutorials, we mainly use the official test sets of several standard datasets as our validation data. The reason we use the given test sets as validation data for the tutorials is that is an easy way to make sure that we all work with the same split and report results using the same data. However, in a proper machine learning setup, your validation set should be separate from the test set, so you can tune the model/parameters on the validation set and then check the final performance in the test set. Thus, even though the variables are `x_test` and `y_test`, they represent our validation set.

Let's first load the dataset and visualise some examples:

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Load CIFAR-10 dataset (train=True loads the training set, download=True downloads if not found)
transform = transforms.Compose([
    transforms.ToTensor()
])

# the data, shuffled and split between train and test sets
# Here we are using the official test set as our validation set, in further
# tutorials, test and validation splits will be explained properly.
# Hence, even though the variables are `x_test` and `y_test`, they represent our validation set
train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

X_train = train_set.data       # NumPy array of shape (50000, 32, 32, 3)
y_train = np.array(train_set.targets)

X_test = test_set.data
y_test = np.array(test_set.targets)

print('Image shape: {0}'.format(X_train.shape[1:]))
print('Total number of training samples: {0}'.format(X_train.shape[0]))
print('Total number of validation samples: {0}'.format(X_test.shape[0]))

# Visualize some examples
N = 5
start_val = 0  # pick an element for the code to plot the following N**2 values
fig, axes = plt.subplots(N, N, figsize=(8, 8))
class_names = train_set.classes  # List of class names: ['airplane', 'automobile', ..., 'truck']

for row in range(N):
    for col in range(N):
        idx = start_val + row + N * col
        axes[row, col].imshow(X_train[idx])
        label_idx = y_train[idx]
        axes[row, col].set_title(class_names[label_idx])
        axes[row, col].set_xticks([])
        axes[row, col].set_yticks([])

fig.subplots_adjust(hspace=0.6)
plt.show()

CIFAR10 contains 50,000 32x32 color training images, labeled over 10 categories and 10,000 test images. As mentioned, we use those test images as our validation set in this tutorial. We need to preprocess the images for the 1D neural network defined in the previous tutorial:

In [ ]:
# Convert to numpy arrays
X_train = train_set.data.astype('float32') / 255.0  # Normalize to [0,1]
X_test = test_set.data.astype('float32') / 255.0

# Flatten the images to shape (N, 3072)
X_train_flatten = X_train.reshape(X_train.shape[0], -1)  # shape (50000, 3072)
X_test_flatten = X_test.reshape(X_test.shape[0], -1)     # shape (10000, 3072)

print('New X_train shape:', X_train_flatten.shape)

y_train = np.array(train_set.targets)
y_test = np.array(test_set.targets)

print('Y_train shape:', y_train.shape)

# Convert data to PyTorch tensors (labels already in integer format)
X_train_tensor = torch.tensor(X_train_flatten, dtype=torch.float32)
Y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_test_tensor = torch.tensor(X_test_flatten, dtype=torch.float32)
Y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Dataset and DataLoader
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

Now, we are ready to define the Multi-layer Perceptron model and train it.

In [ ]:
import torch.nn as nn
from torch.optim import RMSprop

# Define the model
class FullyConnectedNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(3072, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        return self.model(x)

# Instantiate model, loss function, and optimizer
model = FullyConnectedNet()
criterion = nn.CrossEntropyLoss()
optimizer = RMSprop(model.parameters(), lr=1e-4)

# Training loop
for epoch in range(20):
    model.train()
    for xb, yb in train_loader:
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} complete")

# Evaluation
model.eval()
correct = 0
total = 0
loss_total = 0.0
with torch.no_grad():
    for xb, yb in test_loader:
        preds = model(xb)
        loss_total += criterion(preds, yb).item()
        predicted = torch.argmax(preds, dim=1)
        correct += (predicted == yb).sum().item()
        total += yb.size(0)

print("Validation loss:", loss_total / len(test_loader))
print("Validation accuracy:", correct / total)


### Problem Definition

In this exercise, you are asked to test several CNN architectures in the code provided below. Do not modify the optimizer, loss used or parameters related to the training such as the learning rate, they will be investigated in future tutorials. You must focus on the architecture itself: number of convolutional layers, number of filters in every layer, activation functions, pooling operators, among others. Batch Normalization and Dropout layers, which are quite used in CNN architectures, will be also investigated in a future tutorial so you do not have to discuss them.


**Report**:
*   Present a bar figure with the training and validation accuracies for different design choices. Discuss only the parameters that have a significant influence on the network's performance. Explain any discrepancy between training and validation accuracies.
*   Present a sketch that introduces your best architecture. See some examples on how to display networks in [cv-tricks' blog](https://cv-tricks.com/cnn/understand-resnet-alexnet-vgg-inception/).


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
import numpy as np

# the data, shuffled and split between train and val sets
# Here we are using the official test set as our validation set, in further
# tutorials, test and validation splits will be explained properly.
# Hence, even though the variables are `x_test` and `y_test`, they represent our validation set
train_dataset = CIFAR10(root='./data', train=True, download=True, transform=ToTensor())
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=ToTensor())

x_train, y_train = train_dataset.data, train_dataset.targets
x_test, y_test = test_dataset.data, test_dataset.targets

print('Image shape:', x_train.shape[1:])
print('Total number of training samples:', len(train_dataset))
print('Total number of validation samples:', len(test_dataset))

# Convert to float32 and normalize
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Convert to PyTorch tensors
x_train = torch.from_numpy(x_train).permute(0, 3, 1, 2)  # NHWC to NCHW
x_test = torch.from_numpy(x_test).permute(0, 3, 1, 2)
y_train_class = torch.tensor(y_train)
y_test_class = torch.tensor(y_test)

# Create DataLoaders
train_loader = DataLoader(TensorDataset(x_train, y_train_class), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(x_test, y_test_class), batch_size=32, shuffle=False)


# TODO: Define your architecture here
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        # ...

    def forward(self, x):
        # ...
        return x

model = Model()
print(model)

# initiate RMSprop optimizer
optimizer = optim.RMSprop(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(20):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        test_loss += criterion(output, target).item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)
accuracy = 100. * correct / len(test_loader.dataset)

print('Validation loss:', test_loss)
print('Validation accuracy:', accuracy)


---
---

## Task 2: Regression

Now we face a regression task instead of a classification problem. Loss function, activations, and dataset will change in the following task. Thus, instead of having one vector with the probabilities of each class, in this regression problem, the output is a single scalar.

For this second task, we chose the task of estimating house prices based on input images. To get the data run the following script, which clones Ahmed and Moustafa’s [repository](https://github.com/emanhamed/Houses-dataset) into colmap.

In [ ]:
!git clone https://github.com/emanhamed/Houses-dataset
%cd /content/Houses-dataset/Houses\ Dataset

This dataset contains four images of the house (kitchen, frontal, bedroom and bathroom), and attributes (number of bedrooms, number of bathrooms, zip code...). For our exercise, we only use the images of the house. We start with front door images.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

house_section = 'frontal' # select between: kitchen, frontal, bedroom or bathroom
print('We have selected {} images. You can switch to the kitchen, frontal, bedroom or bathroom images by changing house_section variable.'.format(house_section))
images = []
for i_im in range(1, 536):
  image = cv2.imread(str(i_im)+'_'+house_section+'.jpg')
  image = cv2.resize(image, (64, 64))
  images.append(image)

labels = []
f = open('HousesInfo.txt', "r")
for x in f:
  label = (x).split(' ')[-1].split('\n')[0]
  labels.append(label)

# Let's visualize some examples
N=3
start_val = 0 # pick an element for the code to plot the following N**2 values
fig, axes = plt.subplots(N,N)
for row in range(N):
  for col in range(N):
    idx = start_val+row+N*col
    tmp = cv2.cvtColor(images[idx],cv2.COLOR_BGR2RGB)
    axes[row,col].imshow(tmp, cmap='gray')
    fig.subplots_adjust(hspace=0.5)
    target = int(labels[idx])
    axes[row,col].set_title(str(target) + '$')
    axes[row,col].set_xticks([])
    axes[row,col].set_yticks([])

Prepare the dataset for training the model:

In [ ]:
import numpy as np

np.random.seed(1234)

images = np.asarray(images).astype('float32')
labels = np.asarray(labels).astype('float32')

# Normalize the image
max_price = labels.max()
images /= 255.
labels /= max_price

random_idx = np.random.permutation(len(images))
images = images[random_idx]
labels = labels[random_idx]

split_size_val = int(0.8*len(images))
X_train, X_val = images[:split_size_val], images[split_size_val+1:]
Y_train, Y_val = labels[:split_size_val], labels[split_size_val+1:]

# Print shape of training and val images
print('X_train image shape: {0}'.format(X_train.shape))
print('X_val image shape: {0}'.format(X_val.shape))

# Print shape of training and val labels
print('Y_train labels shape: {0}'.format(Y_train.shape))
print('Y_val labels shape: {0}'.format(Y_val.shape))

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

# Set random seed for reproducibility
np.random.seed(1234)  # For numpy operations we might still use

# Convert to PyTorch tensors and adjust dimensions
# PyTorch expects images in (N, C, H, W) format
images_tensor = torch.from_numpy(np.asarray(images)).float().permute(0, 3, 1, 2)  # NHWC to NCHW
labels_tensor = torch.from_numpy(np.asarray(labels)).float()

# Normalize the data
max_price = labels_tensor.max()
images_tensor = images_tensor / 255.0  # Normalize pixel values
labels_tensor = labels_tensor / max_price  # Normalize prices

# Split into training and validation sets
split_size_val = int(0.8 * len(images_tensor))
X_train = images_tensor[:split_size_val]
X_val = images_tensor[split_size_val+1:]
Y_train = labels_tensor[:split_size_val]
Y_val = labels_tensor[split_size_val+1:]

# Create PyTorch Datasets
train_dataset = TensorDataset(X_train, Y_train)
val_dataset = TensorDataset(X_val, Y_val)

# You can optionally create DataLoaders for batching
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Print shape of training and val images
print('X_train image shape:', X_train.shape)
print('X_val image shape:', X_val.shape)

# Print shape of training and val labels
print('Y_train labels shape:', Y_train.shape)
print('Y_val labels shape:', Y_val.shape)

### Problem Definition

Similar to the previous task, you are asked to design a CNN architecture able to perform the estimation of house prices based on the `frontal` house image. Design a new model by changing parameters such as the number of convolutional layers, activation functions, strides, or pooling operators, among others.

In [ ]:
torch.manual_seed(1234)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# TODO: Define here your architecture
class HousePriceModel(nn.Module):
    def __init__(self):
        super(HousePriceModel, self).__init__()
        # Define your layers here
        pass

    def forward(self, x):
        # Define your forward pass here
        return x

model = HousePriceModel()
print(model)

# Define loss function - MAPE needs custom implementation in PyTorch
def mean_absolute_percentage_error(y_pred, y_true):
    return torch.mean(torch.abs((y_true - y_pred) / y_true)) * 100

optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
print("[INFO] training model...")
for epoch in range(100):
    model.train()
    train_loss = 0.0

    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = mean_absolute_percentage_error(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            val_loss += mean_absolute_percentage_error(outputs.squeeze(), targets).item()

    print(f"Epoch {epoch+1}/100")
    print(f"Train MAPE: {train_loss/len(train_loader):.4f}%")
    print(f"Val MAPE: {val_loss/len(val_loader):.4f}%")

The metric used in this problem to evaluate the performance is the same we used for training the model, the mean absolute percentage error. Mean absolute percentage error is defined as $\frac{100}{n} \sum_n \frac{|\hat{y} - y|}{|y|}$ where $y$ is the ground-truth, $\hat{y}$ is the estimation of the model and `n` the number of elements in the set we are evaluating.

In [ ]:
model.eval()
with torch.no_grad():
    prices_predicted = model(X_val).squeeze()
    diff = ((prices_predicted - Y_val) / Y_val) * 100
    error_mean = torch.mean(torch.abs(diff)).item()

print("Predicting house prices - Estimation Error: {:.2f}%".format(error_mean))

**Report**:


*   Propose a CNN architecture that has an estimation error in the validation set below 75%.
*   Present a figure showing the training and validation loss vs the number of training epochs for different architectural design choices. Discuss the gap between the training and validation loss depending on the proposed architecture.
*   Report a table with results when using any of the other images from the house (kitchen, bedroom, and bathroom).